# Logistic classifier over bag of words approach

In [1]:
import torch
import re
import random

### Collect dataset
Resources:
- [Wikiquote: Taras Hryhorovych Shevchenko](https://uk.wikiquote.org/wiki/%D0%A8%D0%B5%D0%B2%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A2%D0%B0%D1%80%D0%B0%D1%81_%D0%93%D1%80%D0%B8%D0%B3%D0%BE%D1%80%D0%BE%D0%B2%D0%B8%D1%87)
- [Wikiquote: Aleksandr Sergeyevich Pushkin](https://ru.wikiquote.org/wiki/%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%A1%D0%B5%D1%80%D0%B3%D0%B5%D0%B5%D0%B2%D0%B8%D1%87_%D0%9F%D1%83%D1%88%D0%BA%D0%B8%D0%BD)

In [2]:
train = [
    ("В своїй хаті своя й правда, і сила, і воля.", "ua"),
    ("Якби ви вчились так, як треба, то й мудрость би була своя.", "ua"),
    ("Учітесь, читайте, і чужому научайтесь, й свого не цурайтесь.", "ua"),
    ("Обніміте ж, брати мої, найменшого брата.", "ua"),
    ("І на оновленій землі врага не буде, супостата, а буде син, і буде мати, і будуть люде на землі.", "ua"),
    ("І ми не ми, і я не я.", "ua"),
    ("І премудрих немудрі одурять.", "ua"),
    ("Кайданами міняються, правдою торгують.", "ua"),
    ("Караюсь, мучуся… але не каюсь.", "ua"),
    ("Ми просто йшли; у нас нема зерна неправди за собою.", "ua"),
    ("Велике щастя буть вольним чоловіком: робиш, що хочеш, ніхто тебе не спинить.", "ua"),
    ("Це правда, що окроме Бога і чорта в душі нашій єсть ще щось таке, таке страшне, що аж холод іде по серцеві, як хоч трошки його розкриєш", "ua"),
    ("Хто не журиться, не плаче, то той ніколи й не радіє.", "ua"),
    ("Лучше любить і робить, аніж писать і говорить.", "ua"),
    ("Мати, всюди однакова мати.", "ua"),
    
    
    
    ("Вдохновение — это умение приводить себя в рабочее состояние.", "ru"),
    ("Во всяком случае, в аду будет много хорошеньких, там можно будет играть в шарады.", "ru"),
    ("Должно стараться иметь большинство на своей стороне: не оскорбляйте же глупцов.", "ru"),
    ("Желудок просвещенного человека имеет лучшие качества доброго сердца: чувствительность и благодарность.", "ru"),
    ("Зависть — сестра соревнования, следственно из хорошего роду.", "ru"),
    ("Зачем кусать нам груди кормилицы нашей; потому что зубки прорезались?", "ru"),
    ("Злы только дураки и дети.", "ru"),
    ("Не откладывай до ужина того, что можешь съесть за обедом.", "ru"),
    ("Не приведи Бог видеть русский бунт, бессмысленный и беспощадный!", "ru"),
    ("Нет ничего безвкуснее долготерпения и самоотверженности.", "ru"),
    ("Первая любовь всегда является делом чувствительности.", "ru"),
    ("Поэзия выше нравственности — или по крайней мере совсем иное дело.", "ru"),
    ("Поэзия, прости господи, должна быть глуповата.", "ru"),
    ("Разберись, кто прав, кто виноват, да обоих и накажи.", "ru"),
    ("Точность — вежливость поваров.", "ru"),
]

test = [
    ("Так от, бач, живу, учусь, нікому не кланяюсь і нікого не боюсь, окроме Бога.", "ua"),
    ("Коли мене неволя і горе не побороло, то сам я не звалюся.", "ua"),
    ("Пустив я їх у люди, а до ції пори ще ніхто й спасибі не сказав.", "ua"),
    ("Бачиш, у мене давно вже думка заворушилась перевести його, те слово, на наш милий, на наш любий український язик.", "ua"),
    ("Я читаю його без вивчення, щодня і щогодини.", "ua"),
    
    ("Они нужны в некоторых только случаях, но и тут можно без них обойтись, а они привыкли всюду соваться.", "ru"),
    ("Я пишу для себя, а печатаю для денег.", "ru"),
    ("В некотором азиатском народе мужчины каждый день, восстав от сна, благодарят Бога, создавшего их не женщинами.", "ru"),
    ("Разве у хорошеньких женщин должен быть характер?", "ru"),
    ("Европа в отношении к России всегда была столь же невежественна, как и неблагодарна.", "ru"),
]

targets = ["ua", "ru"]
target_values = {l: i for i, l in enumerate(targets)}
random.shuffle(train)

### Sentence preprocessing

In [3]:
def process_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^\w ]", "", sentence)
    tokens = sentence.split()
    return tokens

In [4]:
for s, lang in test:
    print(process_sentence(s))

['так', 'от', 'бач', 'живу', 'учусь', 'нікому', 'не', 'кланяюсь', 'і', 'нікого', 'не', 'боюсь', 'окроме', 'бога']
['коли', 'мене', 'неволя', 'і', 'горе', 'не', 'побороло', 'то', 'сам', 'я', 'не', 'звалюся']
['пустив', 'я', 'їх', 'у', 'люди', 'а', 'до', 'ції', 'пори', 'ще', 'ніхто', 'й', 'спасибі', 'не', 'сказав']
['бачиш', 'у', 'мене', 'давно', 'вже', 'думка', 'заворушилась', 'перевести', 'його', 'те', 'слово', 'на', 'наш', 'милий', 'на', 'наш', 'любий', 'український', 'язик']
['я', 'читаю', 'його', 'без', 'вивчення', 'щодня', 'і', 'щогодини']
['они', 'нужны', 'в', 'некоторых', 'только', 'случаях', 'но', 'и', 'тут', 'можно', 'без', 'них', 'обойтись', 'а', 'они', 'привыкли', 'всюду', 'соваться']
['я', 'пишу', 'для', 'себя', 'а', 'печатаю', 'для', 'денег']
['в', 'некотором', 'азиатском', 'народе', 'мужчины', 'каждый', 'день', 'восстав', 'от', 'сна', 'благодарят', 'бога', 'создавшего', 'их', 'не', 'женщинами']
['разве', 'у', 'хорошеньких', 'женщин', 'должен', 'быть', 'характер']
['европа'

### Create words vocabulary

In [5]:
vocabulary = {}
for sentence, _ in train + test:
    for word in process_sentence(sentence):
        if word not in vocabulary:
            vocabulary[word] = len(vocabulary)

In [6]:
input_size = len(vocabulary)
output_size = len(target_values)

### Transform sentense into "bag of words"

In [7]:
def sentence_to_vector(sentence):
    tokens = process_sentence(sentence)
    vector = [0] * input_size
    for t in tokens:
        vector[vocabulary[t]] += 1
    
    return vector

### Simple log regression with pytorch

In [8]:
class LogReg(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return torch.softmax(self.linear(x), 1)
    
model = LogReg(input_size, output_size)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
def evaluate():
    wrong = []
    with torch.no_grad(): 
        for idx, (sentence, lang) in enumerate(test):
            input_value = torch.tensor([sentence_to_vector(sentence)]).float()
            target_value = torch.tensor([target_values[lang]])
            predict = model(input_value)
            predict_lang = targets[predict.max(1)[1].item()]
            
            if lang != predict_lang:
                wrong.append((sentence, lang, predict, predict_lang))
            
            print(idx, sentence)
            print(f"probabilities: {predict.tolist()}")
            print(f"real: {lang}")
            print(f"predict: {predict_lang}")
            print()
            
    return wrong


### Evaluate before train

In [10]:
evaluate();

0 Так от, бач, живу, учусь, нікому не кланяюсь і нікого не боюсь, окроме Бога.
probabilities: [[0.49442726373672485, 0.5055727362632751]]
real: ua
predict: ru

1 Коли мене неволя і горе не побороло, то сам я не звалюся.
probabilities: [[0.48034000396728516, 0.5196600556373596]]
real: ua
predict: ru

2 Пустив я їх у люди, а до ції пори ще ніхто й спасибі не сказав.
probabilities: [[0.46988657116889954, 0.5301134586334229]]
real: ua
predict: ru

3 Бачиш, у мене давно вже думка заворушилась перевести його, те слово, на наш милий, на наш любий український язик.
probabilities: [[0.5766773223876953, 0.42332273721694946]]
real: ua
predict: ua

4 Я читаю його без вивчення, щодня і щогодини.
probabilities: [[0.48903608322143555, 0.5109638571739197]]
real: ua
predict: ru

5 Они нужны в некоторых только случаях, но и тут можно без них обойтись, а они привыкли всюду соваться.
probabilities: [[0.5373398661613464, 0.46266019344329834]]
real: ru
predict: ua

6 Я пишу для себя, а печатаю для денег.
pr

### Model trainig

In [11]:
epoch = 100

for e in range(epoch):
    for sentence, lang in train:
        optimizer.zero_grad()
        
        input_value = torch.tensor([sentence_to_vector(sentence)]).float()
        target_value = torch.tensor([target_values[lang]])
        
        out = model(input_value)
        loss = criterion(out, target_value)
        loss.backward()
        optimizer.step()

### Evaluate model results and save wrong predics

In [12]:
wrong = evaluate();

0 Так от, бач, живу, учусь, нікому не кланяюсь і нікого не боюсь, окроме Бога.
probabilities: [[0.9433287978172302, 0.056671142578125]]
real: ua
predict: ua

1 Коли мене неволя і горе не побороло, то сам я не звалюся.
probabilities: [[0.9469779133796692, 0.053022027015686035]]
real: ua
predict: ua

2 Пустив я їх у люди, а до ції пори ще ніхто й спасибі не сказав.
probabilities: [[0.7978948950767517, 0.20210511982440948]]
real: ua
predict: ua

3 Бачиш, у мене давно вже думка заворушилась перевести його, те слово, на наш милий, на наш любий український язик.
probabilities: [[0.4830685257911682, 0.5169314742088318]]
real: ua
predict: ru

4 Я читаю його без вивчення, щодня і щогодини.
probabilities: [[0.7895163297653198, 0.21048370003700256]]
real: ua
predict: ua

5 Они нужны в некоторых только случаях, но и тут можно без них обойтись, а они привыкли всюду соваться.
probabilities: [[0.10279274731874466, 0.8972072601318359]]
real: ru
predict: ru

6 Я пишу для себя, а печатаю для денег.
prob

### Results understanding
- What are the most influence words for each language?
- Why we get wrong resluts on test set?

In [13]:
weigths = next(model.linear.parameters())
most_important_len = 10
words_importance = {}
for word, idx in vocabulary.items():
    words_importance[word] = {
        "lang": targets[weigths[:, idx].argmax().item()],
        "importance": weigths[:, idx].max().item()
    }

for lang in targets:
    print(lang.upper())
    lang_words = list(filter(lambda v: v[1]["lang"] == lang, words_importance.items()))
    most_important = sorted(lang_words, key=lambda v: v[1]["importance"], reverse=True)
    print(list(map(lambda v: (v[0], v[1]["importance"]), most_important[:most_important_len])))
    print()

UA
[('і', 0.7483457326889038), ('мати', 0.4274558424949646), ('не', 0.31153225898742676), ('й', 0.3012298345565796), ('ми', 0.27373555302619934), ('ж', 0.24648517370224), ('кайданами', 0.2414771318435669), ('правдою', 0.23094195127487183), ('то', 0.22950570285320282), ('караюсь', 0.22051571309566498)]

RU
[('и', 0.5791085362434387), ('вежливость', 0.2346676141023636), ('поэзия', 0.22842636704444885), ('что', 0.2220650464296341), ('будет', 0.21561896800994873), ('же', 0.20251421630382538), ('ужина', 0.2002698928117752), ('умение', 0.19408069550991058), ('поваров', 0.19223856925964355), ('кто', 0.18930502235889435)]



In [14]:
for sentence, lang, predict, predict_lang in wrong:
    print(sentence)
    print(f"probabilities: {predict.tolist()}")
    print(f"real: {lang}")
    print(f"predict: {predict_lang}")
    tokens = process_sentence(sentence)
    tokens = sorted(tokens, key=lambda t: words_importance[t]["importance"], reverse=True)
    for t in tokens:
        print(t, words_importance[t])
    print()

Бачиш, у мене давно вже думка заворушилась перевести його, те слово, на наш милий, на наш любий український язик.
probabilities: [[0.4830685257911682, 0.5169314742088318]]
real: ua
predict: ru
у {'lang': 'ua', 'importance': 0.17231599986553192}
на {'lang': 'ru', 'importance': 0.055308520793914795}
на {'lang': 'ru', 'importance': 0.055308520793914795}
український {'lang': 'ru', 'importance': 0.05167703330516815}
слово {'lang': 'ua', 'importance': 0.05153817683458328}
його {'lang': 'ua', 'importance': 0.04777233675122261}
перевести {'lang': 'ru', 'importance': 0.04253116995096207}
любий {'lang': 'ru', 'importance': 0.04146210849285126}
наш {'lang': 'ua', 'importance': 0.03705056756734848}
наш {'lang': 'ua', 'importance': 0.03705056756734848}
милий {'lang': 'ua', 'importance': 0.0234699547290802}
думка {'lang': 'ua', 'importance': 0.01854587346315384}
те {'lang': 'ru', 'importance': 7.054954767227173e-05}
язик {'lang': 'ru', 'importance': -7.581710815429688e-05}
бачиш {'lang': 'ua', 'impo